In [ ]:
%matplotlib notebook

import h5py
import matplotlib.pyplot as plt
from numpy import *
import pandas as pd
import os
import sys
import scipy.optimize as opt
from typing import Dict, Any
import recordclass.recordclass as rc

In [ ]:
def get_ivars(iteration: h5py._hl.group.Group, *ivar_names: str) -> Dict[str, Any]:
    """
    Gets the values of independent variables for the iteration that was passed in
    
    Args:
        iteration: the data group corresponding to an iteration
        *ivar_names: list of names of independent variables
    Returns:
        ivar_vals : map of ivar_names to their values
    """
    return {name: iteration[f"variables/{name}"][()] for name in ivar_names}

In [ ]:
h5file = h5py.File('results.hdf5',mode='r+')

#Find number of measurements per experiment
measurements = h5file['settings/experiment/measurementsPerIteration'][()]+1
num_its = len(h5file['iterations'])

#Find independent variable names and values
indep_vars = {}
if len(h5file['iterations'])> 1:
    for variable in h5file['settings/experiment/independentVariables'].items():
        values = eval(variable[1]['function'][()])
        #print(f"{variable[0]}\n\t{values}")
        if iterable(values):
            indep_vars.update({variable[0]: list(values)})
iVars = list(indep_vars.keys())

# map iterations to independent variable values
iterations = pd.DataFrame(columns=["iteration"]+list(indep_vars.keys()))
for iteration in h5file['iterations'].items():
    ivar_vals = get_ivars(iteration[1],*list(indep_vars.keys()))
    ivar_vals.update({"iteration": int(iteration[0])})
    iterations = iterations.append(pd.DataFrame(ivar_vals, index = [iteration[0]]))

iterations = iterations.sort_values("iteration")

# Hamamatsu

In [ ]:
from HamamatsuH5 import HMROI, set_frame_grabber_region
import HamamatsuH5
fg = set_frame_grabber_region(h5file)
    
width = fg['right']-fg['left']
height = fg['bottom']-fg['top']

roi = HMROI(width,height,dic = {
    "left" : 5,
    "right" : 9,
    "top" : 3,
    "bottom" : 6
})

In [ ]:
shots_per_measurement = int(h5file['/settings/experiment/LabView/camera/shotsPerMeasurement/function'][()])

hm_pix = HamamatsuH5.load_data(h5file,roi)
   
hm_counts = hm_pix.sum(3).sum(3)
mus = hm_counts.mean(1)
ers = hm_counts.std(1)/sqrt(measurements)

In [ ]:
fig,axarr = plt.subplots(len(iterations),1,figsize=(6,4*len(iterations)))
for iteration, row in iterations.iterrows():
    iteration = int(iteration)
    for shot in range(shots_per_measurement):
        axarr[iteration].hist(hm_counts[iteration,:,shot],histtype='step', label = f"shot {shot}",bins=30)
    axarr[iteration].legend()
    axarr[iteration].set_title(str(row))
fig.tight_layout()
fig.show()

In [ ]:
if len(indep_vars) == 0:
    for shot in range(shots_per_measurement):
        print(f"shot {shot} rate : {mus[0,shot]} +/- {ers[0,shot]}")
elif len(indep_vars) == 1:
    xlin = iterations[list(indep_vars.keys())[0]]
    fig,ax = plt.subplots(1,1,figsize=(6,5))
    for shot in range(shots_per_measurement):
        ax.errorbar(xlin, mus[:,shot], yerr=ers[:,shot], label=f"Shot {shot}")
    ax.legend()
    ax.set_ylabel("Counts in ROI")
    ax.set_xlabel(list(indep_vars.keys())[0])
    fig.show()
else:
    print("Many axes, look for purpose made plotting cells")

* Ensure once things get better they can get involved in Madison
* Not isolated
* on slack (check)
* check on classes/TAing